# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Novas Tecnologias em Banco de Dados</b></center></font>
  
<font size="3"><center>Profa. Dra. Sahudy Montenegro González</center></font>

## <center>Projeto Final</center>

*INTEGRANTES*

**Integrante 01**: João Vitor Averaldo Antunes

**RA**: 813979

**Integrante 02**: Rafael Mori Pinheiro

**RA**: 813851

## Packages e imports

<p> Caso queira instalar todas as dependências para rodar o projeto basta executar a célula abaixo! </p>

In [1]:
# %pip install requests
# %pip install requests_html
# %pip install re
# %pip install asyncio
# %pip install nest_asyncio
# %pip install sqlalchemy
# %pip install io
# %pip install IPython.display
# %pip install bs4
# %pip install lxml_html_clean

<p>Nesta seção apresentamos e importamos todos as biblotecas utilizadas no projeto.</p>

In [2]:
import pandas as pd
import requests
import pandas as pd
from requests_html import HTMLSession, AsyncHTMLSession
import re
import asyncio
import nest_asyncio
from sqlalchemy import create_engine
import numpy as np
import logging
from bs4 import BeautifulSoup

nest_asyncio.apply()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

---

### PIB
DataFrame para o PIB_Per_Capita

In [3]:
# Fetch the data.
df_pib_per_capita = pd.read_csv("https://ourworldindata.org/grapher/gdp-per-capita-worldbank.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Fetch the metadata
metadata = requests.get("https://ourworldindata.org/grapher/gdp-per-capita-worldbank.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

# --> Tratamento <-- #
# Exemplo de leitura (caso seja um CSV)
# df_pib = pd.read_csv('pib_per_capita.csv')

# Renomeia a coluna para ficar consistente com o seu modelo
df_pib_per_capita.rename(columns={'ny_gdp_pcap_pp_kd': 'PIB_Per_Capita'}, inplace=True)

# Converte as colunas para os tipos corretos
df_pib_per_capita['Year'] = df_pib_per_capita['Year'].astype(int)
df_pib_per_capita['PIB_Per_Capita'] = pd.to_numeric(df_pib_per_capita['PIB_Per_Capita'], errors='coerce')

# Opcional: remover linhas com valores ausentes ou preenchê-los conforme a estratégia adotada
df_pib_per_capita.dropna(subset=['PIB_Per_Capita'], inplace=True)

display(df_pib_per_capita)


,Entity,Code,Year,PIB_Per_Capita
0,Afghanistan,AFG,2000,1617.8264
1,Afghanistan,AFG,2001,1454.1108
2,Afghanistan,AFG,2002,1774.3087
3,Afghanistan,AFG,2003,1815.9282
4,Afghanistan,AFG,2004,1776.9182
...,...,...,...,...
7058,Zimbabwe,ZWE,2019,3294.8062
7059,Zimbabwe,ZWE,2020,2987.2698
7060,Zimbabwe,ZWE,2021,3184.7854
7061,Zimbabwe,ZWE,2022,3323.1218


### Acesso Educacao
DataFrame para o Acesso_Educacao

In [4]:
# Fetch the data.
df_acesso_educacao = pd.read_csv("https://ourworldindata.org/grapher/learning-outcomes-vs-gdp-per-capita.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Fetch the metadata
metadata = requests.get("https://ourworldindata.org/grapher/learning-outcomes-vs-gdp-per-capita.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

# --> Tratamento <-- #
# Suponha que o dataframe se chame df_educacao
# df_educacao = pd.read_csv('acesso_educacao.csv')

# Converte o ano para inteiro
df_acesso_educacao['Year'] = df_acesso_educacao['Year'].astype(int)

# Converter a coluna de scores para numérico
df_acesso_educacao['harmonized_test_scores'] = pd.to_numeric(df_acesso_educacao['harmonized_test_scores'], errors='coerce')

# Remover valores ausentes (ou aplicar outra estratégia de preenchimento)
df_acesso_educacao.dropna(subset=['harmonized_test_scores'], inplace=True)

# Normaliza os scores para gerar um percentual
min_score = df_acesso_educacao['harmonized_test_scores'].min()
max_score = df_acesso_educacao['harmonized_test_scores'].max()

df_acesso_educacao['Acesso_Educacao'] = ((df_acesso_educacao['harmonized_test_scores'] - min_score) / (max_score - min_score)) * 100

# Se necessário, selecione apenas as colunas de interesse para o merge posterior
df_acesso_educacao = df_acesso_educacao[['Entity', 'Code', 'Year', 'Acesso_Educacao']]

display(df_acesso_educacao)

,Entity,Code,Year,Acesso_Educacao
0,Afghanistan,AFG,2017,18.139005
1,Afghanistan,AFG,2018,18.051639
2,Afghanistan,AFG,2020,18.051639
604,Albania,ALB,2010,33.434287
605,Albania,ALB,2017,44.943048
...,...,...,...,...
58980,Zambia,ZMB,2020,19.276526
59238,Zimbabwe,ZWE,2010,32.337201
59239,Zimbabwe,ZWE,2017,32.989894
59240,Zimbabwe,ZWE,2018,33.040177


### Expectativa de Vida
DataFrame para a Expectativa_Vida

In [5]:
# Fetch the data.
df_expectativa_vida = pd.read_csv("https://ourworldindata.org/grapher/life-expectancy.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Fetch the metadata
metadata = requests.get("https://ourworldindata.org/grapher/life-expectancy.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

# --> Tratamento <-- #
# Suponha que o dataframe se chame df_vida
# df_vida = pd.read_csv('expectativa_vida.csv')

# Renomeia a coluna para ficar consistente com o modelo
df_expectativa_vida.rename(columns={'life_expectancy_0__sex_total__age_0': 'Expectativa_Vida'}, inplace=True)

# Converte o ano para inteiro e a expectativa para numérico
df_expectativa_vida['Year'] = df_expectativa_vida['Year'].astype(int)
df_expectativa_vida['Expectativa_Vida'] = pd.to_numeric(df_expectativa_vida['Expectativa_Vida'], errors='coerce')

# Tratar valores ausentes, se necessário
df_expectativa_vida.dropna(subset=['Expectativa_Vida'], inplace=True)

df_expectativa_vida.to_csv("expectativa.csv")

display(df_expectativa_vida)


,Entity,Code,Year,Expectativa_Vida
0,Afghanistan,AFG,1950,28.1563
1,Afghanistan,AFG,1951,28.5836
2,Afghanistan,AFG,1952,29.0138
3,Afghanistan,AFG,1953,29.4521
4,Afghanistan,AFG,1954,29.6975
...,...,...,...,...
21560,Zimbabwe,ZWE,2019,61.0603
21561,Zimbabwe,ZWE,2020,61.5300
21562,Zimbabwe,ZWE,2021,60.1347
21563,Zimbabwe,ZWE,2022,62.3601


### Taxa de mortalidade
DataFrame para Taxa_Mortalidade

In [6]:
# Fetch the data.
df_taxa_mortalidade = pd.read_csv('taxa_mortalidade.csv')

# --> Tratamento <-- #
# Suponha que o dataframe se chame df_mortalidade
# df_mortalidade = pd.read_csv('taxa_mortalidade.csv')

# Converte o ano para inteiro (a coluna pode estar em minúsculas)
df_taxa_mortalidade['year'] = df_taxa_mortalidade['year'].astype(int)

# Converte a coluna 'val' para numérico; renomeia para Taxa_Mortalidade
df_taxa_mortalidade['Taxa_Mortalidade'] = pd.to_numeric(df_taxa_mortalidade['val'], errors='coerce')

# Se for necessário transformar um número absoluto em percentual, por exemplo:
# df_mortalidade['Taxa_Mortalidade'] = (df_mortalidade['Taxa_Mortalidade'] / df_populacao['Populacao_Total']) * 100

# Remova linhas com valores ausentes (se apropriado)
df_taxa_mortalidade.dropna(subset=['Taxa_Mortalidade'], inplace=True)

# Seleciona as colunas que serão usadas para o merge
df_taxa_mortalidade = df_taxa_mortalidade[['location_name', 'year', 'Taxa_Mortalidade']]

display(df_taxa_mortalidade)

,location_name,year,Taxa_Mortalidade
0,Iran (Islamic Republic of),1950,1.679013e+07
1,Iran (Islamic Republic of),1951,1.718549e+07
2,Iran (Islamic Republic of),1952,1.760677e+07
3,Iran (Islamic Republic of),1953,1.805103e+07
4,Iran (Islamic Republic of),1954,1.851493e+07
...,...,...,...
14683,Bahrain,2017,1.453405e+06
14684,Bahrain,2018,1.473801e+06
14685,Bahrain,2019,1.494478e+06
14686,Bahrain,2020,1.513957e+06


### Médicos por habitante
DataFrame para Medicos_Por_Habitante

In [7]:
df_medicos_por_habitante = pd.read_csv('medicos_por_habitante.csv')

# --> Tratamento <-- #
# Suponha que o dataframe se chame df_medicos
# df_medicos = pd.read_csv('medicos_por_habitante.csv')

# Converte o ano para inteiro; a coluna de ano pode estar com nome 'Period' ou 'Year'
df_medicos_por_habitante['Period'] = df_medicos_por_habitante['Period'].astype(int)

# Converter a coluna com o valor dos médicos para numérico
df_medicos_por_habitante['Medicos_Por_Habitante'] = pd.to_numeric(df_medicos_por_habitante['Value'], errors='coerce')

# Remover valores ausentes, se necessário
df_medicos_por_habitante.dropna(subset=['Medicos_Por_Habitante'], inplace=True)

# Se necessário, selecione apenas as colunas para o merge
df_medicos_por_habitante = df_medicos_por_habitante[['Location', 'Period', 'Medicos_Por_Habitante']]

display(df_medicos_por_habitante)


,Location,Period,Medicos_Por_Habitante
0,Niger,2023,0.38
1,Papua New Guinea,2023,0.61
2,Central African Republic,2023,0.74
3,Chad,2023,0.85
4,Gambia,2023,0.90
...,...,...,...
3411,Republic of Korea,1990,8.12
3412,Oman,1990,8.17
3413,Türkiye,1990,9.04
3414,Ecuador,1990,9.34


### Em Conflito
DataFrame para Em_Conflito

In [8]:
# Fetch the data.
df_em_conflito = pd.read_csv("https://ourworldindata.org/grapher/civilian-and-combatant-deaths-in-armed-conflicts-based-on-where-they-occurred.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Fetch the metadata
metadata = requests.get("https://ourworldindata.org/grapher/civilian-and-combatant-deaths-in-armed-conflicts-based-on-where-they-occurred.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

# ---> Tratamento <--- #
# Supondo que df_conflito seja o dataframe de conflito:
df_em_conflito['total_deaths'] = (df_em_conflito['number_deaths_civilians__conflict_type_all'] +
                               df_em_conflito['number_deaths_unknown__conflict_type_all'] +
                               df_em_conflito['number_deaths_combatants__conflict_type_all'])

def conflict_level(deaths):
    if deaths < 100:
        return 'Baixo'
    elif deaths < 1000:
        return 'Médio'
    else:
        return 'Alto'

df_em_conflito['Em_Conflito'] = df_em_conflito['total_deaths'].apply(conflict_level)

display(df_em_conflito)


,Entity,Code,Year,number_deaths_civilians__conflict_type_all,number_deaths_unknown__conflict_type_all,number_deaths_combatants__conflict_type_all,total_deaths,Em_Conflito
0,Abkhazia,OWID_ABK,1989,0,0,0,0,Baixo
1,Abkhazia,OWID_ABK,1990,0,0,0,0,Baixo
2,Abkhazia,OWID_ABK,1991,0,0,0,0,Baixo
3,Abkhazia,OWID_ABK,1992,0,0,0,0,Baixo
4,Abkhazia,OWID_ABK,1993,0,0,0,0,Baixo
...,...,...,...,...,...,...,...,...
7170,Zimbabwe,ZWE,2019,0,0,0,0,Baixo
7171,Zimbabwe,ZWE,2020,0,0,0,0,Baixo
7172,Zimbabwe,ZWE,2021,0,0,0,0,Baixo
7173,Zimbabwe,ZWE,2022,0,0,0,0,Baixo


### Religião
DataFrame para Religiões

In [9]:
# Lendo o CSV (substitua 'caminho_do_arquivo.csv' pelo caminho correto)
df_religiao = pd.read_csv('religiao.csv')

# Removendo a coluna de footnotes
df_religiao = df_religiao.drop(columns=["Value Footnotes"])

# Mapeamento das religiões para a classificação desejada
mapping = {
    # – Agrupamento de agregados ou totais
    "Total": "Não Informado",  # Total é um agregado, não uma religião específica

    # Secular / Não Religioso
    "Atheist": "Secular/Não Religioso",
    "No Religion": "Secular/Não Religioso",
    "Non-believers": "Secular/Não Religioso",
    "No religion, Agnostic, Atheist": "Secular/Não Religioso",
    "Agnostic": "Secular/Não Religioso",
    "Humanist": "Secular/Não Religioso",

    # Cristão – diversas denominações
    "Catholic": "Cristão",
    "Roman Catholic": "Cristão",
    "Other Catholic": "Cristão",
    "Brasilian Apostolic Catholic": "Cristão",
    "Roman Apostolic Catholic": "Cristão",
    "Old Catholic Church": "Cristão",
    "Greek Catholic": "Cristão",
    "Ukrainian Catholic": "Cristão",
    "Evangelical": "Cristão",
    "Other Evangelical Pentecostal": "Cristão",
    "Other Traditional Evangelical": "Cristão",
    "Jehovah Witness": "Cristão",
    "Jehovah's Witnesses": "Cristão",
    "Yeshiva's witness": "Cristão",
    "Moravian": "Cristão",
    "Salvation Army": "Cristão",
    "Church of God": "Cristão",
    "Church of God nos": "Cristão",
    "Christian": "Cristão",
    "Other Christians": "Cristão",
    "Brethren": "Cristão",
    "Brethren in Christ": "Cristão",
    "Anglican": "Cristão",
    "Baptist": "Cristão",
    "Bible Baptist": "Cristão",
    "Congregational": "Cristão",
    "Christian nfd": "Cristão",
    "Presbyterian": "Cristão",
    "Presbyterian and Reformed": "Cristão",
    "Protestant": "Cristão",
    "Other Protestant": "Cristão",
    "Methodist": "Cristão",
    "Pentecostal": "Cristão",
    "Seventh Day Adventist": "Cristão",
    "Adventist": "Cristão",
    "Lutheran": "Cristão",
    "United Church": "Cristão",
    "Non-denominational": "Cristão",
    "Church of Christ": "Cristão",
    "Druze": "Outros",  # Embora alguns considerem o Druze parte do espectro do Oriente Médio, aqui optei por "Outros"
    "Latter Day Saints": "Cristão",
    "Mormon": "Cristão",
    "Latter Day Saints (Mormon)": "Cristão",
    "Church of Jesus Christ of Latter-day Saints": "Cristão",
    "Church of Nazarene": "Cristão",
    "Coptic Orthodox": "Cristão",
    "Eastern Orthodox": "Cristão",
    "Orthodox": "Cristão",
    "Oriental Orthodox": "Cristão",
    "Antiochian Orthodox Christian": "Cristão",
    "Armenian Apostolic": "Cristão",
    "Armenian Orthodox": "Cristão",
    "Greek Oriental": "Cristão",
    "Greek Orthodox": "Cristão",
    "Russian Orthodox": "Cristão",
    "Ukrainian Orthodox": "Cristão",
    "Romanian Orthodox": "Cristão",
    "Serbian Orthodox": "Cristão",
    "Swedenborgian (New Church)": "Cristão",
    "Assyrian Apostolic": "Cristão",
    "Oriental Christian": "Cristão",
    "Christengmeinschaft": "Cristão",
    "Church of England": "Cristão",
    "Mennonite": "Cristão",
    "New Apostolic": "Cristão",
    "Shalom": "Cristão",  # Se aparecer algum termo similar
    "Unitarian": "Outros",  # Unitarian Universalism pode ser classificado como "Outros"
    "Unity-New Thought-Pantheist": "Nova Era",
    "Worldwide Church of God": "Cristão",
    "Brethren Exclusive": "Cristão",
    "Christian Community": "Cristão",
    "Free Christian Community": "Cristão",
    "Assembly of God": "Cristão",
    "Church of Christ Filipinista": "Cristão",
    "Christian Congregation of Brazil": "Cristão",
    "Espiritia": "Cristão",  # Apesar de ser uma doutrina própria, frequentemente está ligada à tradição cristã no Brasil
    "God is Love": "Cristão",
    "Orthodox Catholic": "Cristão",
    "Universal of the Kingdom of God": "Cristão",
    "Messianica": "Cristão",
    "New Testament Church of God": "Cristão",
    "Armeno-Gregorian": "Cristão",
    "Antiochian Orthodox Christian": "Cristão",
    "Associated Gospel": "Cristão",
    "Christadelphian": "Cristão",
    "Christian or Plymouth Brethren": "Cristão",
    "Christian Reformed Church": "Cristão",
    "Church of Nazarene": "Cristão",
    "Doukhobors": "Cristão",
    "Evangelical Free Church": "Cristão",
    "Evangelical Missionary Church": "Cristão",
    "Mission de l'Esprit Saint": "Cristão",
    "Quakers": "Cristão",
    "Serbian Orthodox": "Cristão",
    "Vineyard Christian Fellowship": "Cristão",
    "Other Methodist Churches": "Cristão",
    "Other Orthodox Churches": "Cristão",
    "Other Reformed Church": "Cristão",
    "Apostolic Christian Church": "Cristão",
    "Canadian Reformed Church": "Cristão",
    "Charismatic Renewal": "Cristão",
    "Christian Congregational": "Cristão",
    "Christian and Mission Alliance": "Cristão",
    "Dutch Reformed": "Cristão",
    "Free Methodists": "Cristão",
    "Interdenominational": "Cristão",
    "Protestant nfd": "Cristão",
    "Reorganized Church of Latter-day Saints": "Cristão",
    "Worldwide Church of God": "Cristão",
    "Brethren in Christ": "Cristão",
    "Christians, n.i.e.": "Cristão",
    "Canadian and American Reformed Church": "Cristão",
    "Hutterite": "Cristão",
    "Standard Church": "Cristão",
    "Polish Catholic Church": "Cristão",
    "Apostolic": "Cristão",
    "Cook Islands Christian Church": "Cristão",
    "Holy Spirit": "Cristão",
    "Calvinist": "Cristão",
    "Armenian church": "Cristão",
    "Maronite church": "Cristão",
    "Czechoslovak Hussite Church": "Cristão",
    "Orthodox Church in the Czech Lands": "Cristão",
    "Evangelical United Brethren Church": "Cristão",
    "Silesian Evangelical Church": "Cristão",
    "Other Evangelical Churches": "Cristão",
    "Old Believers": "Cristão",
    "Charismatic Episcopal Church": "Cristão",
    "Christian Free Congregations": "Cristão",
    "Church of Charismatic Fellowship": "Cristão",
    "Church of Christian Fellowship": "Cristão",
    "Russian Catholic Orthodox Church": "Cristão",
    "Ukrainian Greek-Catholic Church": "Cristão",
    "Full Gospel Church": "Cristão",
    "Other Apostolic Churches": "Cristão",
    "Free Church in Finland": "Cristão",
    "Other Lutheran Churches": "Cristão",
    "Roman Catholic Church in Finland": "Cristão",
    "The Greek Orthodox Church in Finland": "Cristão",
    "Christian Orthodox": "Cristão",
    "Church of Ireland": "Cristão",
    "Lapsed Roman Catholic": "Cristão",
    "Church of God of Prophecy": "Cristão",
    "Other Church of God": "Cristão",
    "Church of God in Jamaica": "Cristão",
    "African Instituted Churches": "Cristão",
    "Church of the Foursquare Gospel": "Cristão",
    "All Nations Baptist": "Cristão",
    "Kempsville Presbyterian Church": "Cristão",
    "Protestant Reformed": "Cristão",
    "Protestant Lutheran": "Cristão",
    "Evangelical Lutherans": "Cristão",
    "Evangelical Reformists": "Cristão",
    "Word of Faith": "Cristão",
    "Charismatic Evangelical Church": "Cristão",
    "All Gospel Church": "Cristão",
    "Neuapostolische Kirche": "Cristão",
    "Non-denominational Christian Churches": "Cristão",
    "Ethnic Churches": "Cristão",
    "Christian Tamil": "Cristão",
    "Mission Salut et Guerison": "Cristão",
    "Bible Centered Fellowship": "Cristão",
    "Bread of Life Ministries": "Cristão",
    "Charismatic Full Gospel Ministries": "Cristão",
    "Christ Faith Fellowship": "Cristão",
    "Christ the Living Stone Fellowship": "Cristão",
    "Christ's Commission Fellowship": "Cristão",
    "Christian Brethren": "Cristão",
    "Christian Missions": "Cristão",
    "Church Body of Christ Filipinista": "Cristão",
    "Conservative Baptist Association": "Cristão",
    "Crusaders of the Divine Church of Christ Incorporated": "Cristão",
    "Filipino Assemblies Of The First Born Incorporated": "Cristão",
    "Free Believers In Christ Fellowship": "Cristão",
    "Fundamental Grace Gospel Church Of Christ": "Cristão",
    "General Baptist Churches": "Cristão",
    "Good News Christian Churches": "Cristão",
    "I Am Redeemer and Master Evangelical Church Inc.": "Cristão",
    "Iglesia Filipina Independiente": "Cristão",
    "Iglesia Sa Dios Espiritu Santo Incorporated": "Cristão",
    "International One Way Outreach": "Cristão",
    "Jesus Christ Saves Global Outreach": "Cristão",
    "Jesus is Alive Community Incorporated": "Cristão",
    "Jesus is Lord Church": "Cristão",
    "Jesus Reigns Ministries": "Cristão",
    "Jesus The Anointed One Church": "Cristão",
    "Miracle Life Fellowship International": "Cristão",
    "Miracle Revival Church": "Cristão",
    "Philippine Ecumenical Christian Church": "Cristão",
    "Independent Catholic Church": "Cristão",
    "Victory Chapel Christian Fellowship": "Cristão",
    "Victory Christian Fellowship": "Cristão",
    "Missionary Baptist Churches of the Philippines": "Cristão",
    "Evangelic of Augsburg Affiliation": "Cristão",
    "Old Catholic Mariavite Church": "Cristão",
    "Baptist Union of Poland": "Cristão",
    "Other Eastern Christianity": "Cristão",
    "Evangelic Augsburg Confession": "Cristão",
    "Témoin de Jehovah": "Cristão",

    # Muçulmano – termos relacionados ao islamismo
    "Muslim": "Muçulmano",
    "Islam ": "Muçulmano",
    "Islamic Congregation": "Muçulmano",
    "Sunni Muslims": "Muçulmano",
    "Holi Muslim": "Muçulmano",  # se houver
    "Other Muslims": "Muçulmano",
    "Mohamedan": "Muçulmano",
    "Hoi giao - Muslim": "Muçulmano",
    "Qadiani/Ahmadi": "Muçulmano",
    "Sufi": "Muçulmano",
    "Shiite Islam": "Muçulmano",

    # Judaico – termos relacionados ao judaísmo
    "Jewish": "Judaico",
    "Judaism": "Judaico",
    "Yiddish": "Judaico",
    "Mosaic": "Judaico",
    "Karaite": "Judaico",

    # Hindu
    "Hindu": "Hindu",
    "Hare Rama Hare Krishna": "Hindu",
    "Arya Samajist": "Hindu",
    "Bengali": "Hindu",
    "Society for Krishna Consciousness": "Hindu",
    "Kabir Panthis": "Hindu",
    "Marathi & Marathi Hindu": "Hindu",
    "Puranic": "Hindu",
    "Rajput": "Hindu",
    "Tamil and Tamil Hindu": "Hindu",
    "Telegu and Telegu Hindu": "Hindu",
    "Vaish": "Hindu",
    "Vedic": "Hindu",
    "Other Hindu": "Hindu",
    "Sanatanist": "Hindu",
    "Arya Samajist, Other": "Hindu",
    "Telegu Speaking": "Hindu",

    # Budista
    "Buddhist": "Budista",
    "Zen Buddhist": "Budista",
    "Won Buddhist": "Budista",
    "Phat giao - Buddhist": "Budista",
    "Buddhism/ Sikh": "Budista",  # aqui considerei a parte budista

    # Tradicionais (Religiões indígenas, ancestrais ou locais)
    "Other": "Outros",  # caso genérico, se não se encaixar em outra
    "Pagan": "Tradicionais",
    "Badimo": "Tradicionais",
    "Candomble": "Tradicionais",
    "Umbanda": "Tradicionais",
    "Umbanda e Candomblé": "Tradicionais",
    "Animist": "Tradicionais",
    "Traditional": "Tradicionais",
    "Aboriginal spirituality": "Tradicionais",
    "Taara or Earth Believer": "Tradicionais",
    "Estonian Ancient Belief": "Tradicionais",
    "Tribal": "Tradicionais",
    "African Traditional Belief": "Tradicionais",
    "Orisha": "Tradicionais",
    "Druid": "Tradicionais",
    "Mäori Religion": "Tradicionais",
    "Prakriti": "Tradicionais",

    # Nova Era / Espiritualidade
    "Spiritualist": "Nova Era",
    "Unification": "Nova Era",
    "New Age": "Nova Era",
    "Unity-New Thought-Pantheist": "Nova Era",
    "Pantheism": "Nova Era",
    "Wiccan": "Nova Era",
    "Yoga": "Nova Era",
    "Mahikari": "Nova Era",
    
    # Não Informado (falta de especificação ou respostas evasivas)
    "Not Stated": "Não Informado",
    "Not Specified": "Não Informado",
    "Not Declared": "Não Informado",
    "Refused to answer": "Não Informado",
    "Unknown": "Não Informado",
    "Didn't answer": "Não Informado",
    "Not Asked": "Não Informado",
    "Denomination not specified": "Não Informado",
    "None or Refused": "Não Informado",
    "Non-Apostolic": "Não Informado",  # se não houver outra informação

    # Outros – para os casos que não se encaixam ou são muito específicos
    "Other Religions": "Outros",
    "Other Religions and Persuasions": "Outros",
    "Other Non-Christians": "Outros",
    "Chinese Religions": "Outros",
    "Japanese Religions": "Outros",
    "Zoroastrianism": "Outros",
    "Zoroastrian": "Outros",
    "Gnostic": "Outros",
    "Jains": "Outros",
    "Shinto": "Outros",
    "Unitarian": "Outros",
    "Other Eastern religions": "Outros",
    "Celestial": "Outros",
    "Harriste": "Outros",
    "Oriental": "Outros",
    "Confucian": "Outros",
    "Revived": "Outros",
    "Te Ran": "Outros",
    "Te koaua": "Outros",
    "Balts Believers": "Outros",
    "Church of Scientology": "Outros",
    "Confucianism/Taoism and Other Traditional Chinese": "Outros",
    "Ahir": "Outros",
    "Ahmadhya": "Outros",
    "Aryan": "Outros",
    "Chinese": "Outros",
    "Historics": "Outros",
    "Orientalists' Cults": "Outros",
    "Saio/Zione": "Outros",
    "Bon": "Outros",
    "Kirati": "Outros",
    "Ratana": "Outros",
    "Ibandlalama Nazaretha": "Outros",
    "Bandla Lama Nazaretha": "Outros",
    "Modekngei": "Outros",
    "Mo'ui Fo'ou 'ia Kalaisi": "Outros",
    "Cao daii - Cao Dai": "Outros",
    "Hoa hao - Hoa Hao": "Outros",
    "Ch'ondo-gyo": "Outros",
    "Taejong-gyo": "Outros",
    "Daesoonjillihoi": "Outros",
    "Revival Centres": "Outros",  # se não for explicitamente cristão
    "Things to Come": "Outros",
    "Other Faiths": "Não Informado"
}
# Tratando valores ausentes (NaN) – mapeando para "Não Informado"
mapping[np.nan] = "Não Informado"

# Se desejar excluir os totais (se for o caso), você pode filtrar:
# df = df[df["Religion"] != "Total"]

# Criando a nova coluna "Classificacao" usando o mapeamento. 
# Caso algum valor não esteja no dicionário, retorna o próprio valor ou outra categoria, conforme necessidade.
df_religiao["Classificacao"] = df_religiao["Religion"].map(mapping).fillna("Outras")

# Visualizando o resultado
display(df_religiao)

print(df_religiao['Religion'].unique())


,Country or Area,Year,Religion,Value,Classificacao
0,Albania,2011,Total,2800138.0,Não Informado
1,Albania,2011,Atheist,69995.0,Secular/Não Religioso
2,Albania,2011,Catholic,280921.0,Cristão
3,Albania,2011,Evangelical,3797.0,Cristão
4,Albania,2011,Muslim,1587608.0,Muçulmano
...,...,...,...,...,...
25947,Zambia,2010,Hindu,189.0,Hindu
25948,Zambia,2010,Muslim,12038.0,Muçulmano
25949,Zambia,2010,Protestant,2915043.0,Cristão
25950,Zambia,2010,Other,72222.0,Outros


['Total' 'Atheist' 'Catholic' 'Evangelical' 'Muslim' 'Orthodox' 'Other'
 'Other Christians' 'Refused to answer' 'Bektashism'
 'Believers without denomination' 'Not Stated' 'Anglican' 'Baptist'
 'Brethren' 'Church of God' 'Hindu' 'Jehovah Witness' 'Jewish' 'Methodist'
 'Pentecostal' 'Presbyterian' 'Rastafarian' 'Roman Catholic'
 'Seventh Day Adventist' 'Other Religions' 'No Religion' 'Not Specified'
 "Baha'i" 'Islam ' 'Moravian' 'Salvation Army' 'Spiritualist'
 'Not Declared' 'Armenian Apostolic' 'Armenian Orthodox' 'Pagan'
 "Yeshiva's witness" 'Molokai' 'Shar-fadinian' 'Adventist' 'Protestant'
 'Other Religions and Persuasions' 'Buddhist' 'Christian'
 'Church of Christ' 'Congregational' 'Druze' 'Eastern Orthodox' 'Judaism'
 'Latter Day Saints' 'Lutheran' 'Sikh' 'United Church' 'Other Protestant'
 "Jehovah's Witnesses" 'Christian nfd' 'Chinese Religions'
 'Presbyterian and Reformed' 'Japanese Religions'
 'Nature and earth based religions' 'Zoroastrianism'
 'Religious belief, nfd and Ina

### Web Scraping 
Fonte dos dados: https://www.worldometers.info/world-population/

In [10]:
import nest_asyncio
import asyncio
import re
import logging
import pandas as pd
import numpy as np
from io import StringIO
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import sys
import warnings

warnings.filterwarnings("ignore")

# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout), logging.FileHandler('scraping.log')]
)
logger = logging.getLogger(__name__)

# Permite o uso de asyncio em ambientes como Jupyter Notebook
nest_asyncio.apply()

async def fetch_population_table(session):
    """
    Obtém a página principal de "Population by Country" e extrai a tabela via pandas.
    Em paralelo, utiliza BeautifulSoup para capturar o link de cada país.
    Retorna um DataFrame com uma coluna adicional "Country_URL".
    """
    url = "https://www.worldometers.info/world-population/population-by-country/"
    try:
        response = await session.get(url)
        # Extrai a tabela com pd.read_html
        tables = pd.read_html(StringIO(response.text))
        pop_table = None
        for table in tables:
            table.columns = [re.sub(r'\s+', ' ', str(col)).strip() for col in table.columns]
            if any("2024" in col and "Population" in col for col in table.columns):
                pop_table = table
                break
        if pop_table is None:
            logger.error("Tabela de população não encontrada na página principal.")
            return None

        # Extraindo os links via BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        table_html = soup.find("table", {"id": "example2"})
        country_links_dict = {}
        if table_html:
            body_rows = table_html.find("tbody").find_all("tr", recursive=False)
            for row in body_rows:
                cols = row.find_all("td", recursive=False)
                if len(cols) < 2:
                    continue
                a_tag = cols[1].find("a", href=True)
                if a_tag:
                    country_name = a_tag.text.strip()
                    relative_url = a_tag['href'].strip()
                    full_url = "https://www.worldometers.info" + relative_url
                    country_links_dict[country_name] = full_url
        else:
            logger.warning("Tabela HTML com id='example2' não foi localizada; os links não serão inseridos.")

        # Renomeia a coluna com o nome do país para facilitar o merge
        original_country_col = "Country (or dependency)"
        if original_country_col in pop_table.columns:
            pop_table.rename(columns={original_country_col: "Country"}, inplace=True)
        else:
            logger.warning(f"Coluna '{original_country_col}' não encontrada.")

        pop_table["Country_URL"] = pop_table["Country"].map(country_links_dict)
        return pop_table

    except Exception as e:
        logger.error(f"Erro ao obter a tabela de população: {str(e)}")
        return None

async def process_population_data():
    """
    Processa a tabela principal para gerar um DataFrame com:
      - Country
      - Populacao_Total (da coluna "Population (2024)")
      - Taxa_Crescimento (da coluna "Yearly Change")
      - Urban_Percent (da coluna "Urban Pop %")
      - Populacao_Urbana e Populacao_Rural (calculadas)
      - Country_URL (link para a página do país)
    """
    session = AsyncHTMLSession()
    try:
        pop_table = await fetch_population_table(session)
        if pop_table is None:
            return None
        
        logger.info(f"Tabela principal extraída com {len(pop_table)} linhas e colunas: {list(pop_table.columns)}")
        
        pop_table.rename(columns={
            "Population (2024)": "Populacao_Total",
            "Yearly Change": "Taxa_Crescimento",
            "Urban Pop %": "Urban_Percent"
        }, inplace=True, errors='ignore')
        
        if "Populacao_Total" in pop_table.columns:
            pop_table["Populacao_Total"] = (
                pop_table["Populacao_Total"]
                .astype(str)
                .str.replace(",", "", regex=False)
                .astype(int)
            )
        
        def process_numeric(val):
            val_str = str(val).strip()
            if val_str.upper() in ["N.A.", "NA", "NAN", ""]:
                return np.nan
            try:
                return float(val_str.replace("%", ""))
            except Exception:
                return np.nan
        
        if "Taxa_Crescimento" in pop_table.columns:
            pop_table["Taxa_Crescimento"] = pop_table["Taxa_Crescimento"].apply(process_numeric)
        if "Urban_Percent" in pop_table.columns:
            pop_table["Urban_Percent"] = pop_table["Urban_Percent"].apply(process_numeric)
        
        if "Populacao_Total" in pop_table.columns and "Urban_Percent" in pop_table.columns:
            pop_table["Populacao_Urbana"] = (
                (pop_table["Populacao_Total"] * pop_table["Urban_Percent"] / 100)
                .round(0)
                .astype("Int64")
            )
            pop_table["Populacao_Rural"] = pop_table["Populacao_Total"] - pop_table["Populacao_Urbana"]
        
        cols_desejadas = ["Country", "Populacao_Total", "Taxa_Crescimento", 
                          "Urban_Percent", "Populacao_Urbana", "Populacao_Rural", "Country_URL"]
        df_population = pop_table[cols_desejadas].copy()
        return df_population
    except Exception as e:
        logger.error(f"Erro crítico no processamento dos dados populacionais: {str(e)}")
        return None
    finally:
        await session.close()

def process_historical_table(df, country_name):
    """
    Dado o DataFrame extraído da página histórica de um país,
    seleciona (se existirem) apenas as colunas:
      - Year
      - Population (população total)
      - (Taxa de) Crescimento – coluna que contenha "Change" e "%" (taxa de crescimento)
      - Urban_Percent – coluna que contenha "Urban" e "%" (percentual de população urbana)
      - Urban_Population – coluna que contenha "Urban" e "Population" (população urbana)
    Calcula ainda:
      - Rural_Population = Population - Urban_Population
    Retorna um DataFrame com as colunas:
       Country, Year, Population, Growth_Rate, Urban_Percent, Urban_Population, Rural_Population
    """
    # Normaliza os nomes das colunas
    df.columns = [re.sub(r'\s+', ' ', str(col)).strip() for col in df.columns]
    
    col_year = None
    col_population = None
    col_growth = None
    col_urban_percent = None
    col_urban_pop = None
    
    for col in df.columns:
        col_lower = col.lower()
        if "year" in col_lower and col_year is None:
            col_year = col
        elif "population" in col_lower and "urban" not in col_lower and "world" not in col_lower and col_population is None:
            col_population = col
        elif "change" in col_lower and "%" in col_lower and col_growth is None:
            col_growth = col
        elif "urban" in col_lower and "%" in col_lower and col_urban_percent is None:
            col_urban_percent = col
        elif "urban" in col_lower and "population" in col_lower and "%" not in col_lower and col_urban_pop is None:
            col_urban_pop = col
            
    if not all([col_year, col_population, col_growth, col_urban_percent, col_urban_pop]):
        logger.warning(f"Para o país {country_name}, nem todas as colunas foram identificadas: "
                       f"Year: {col_year}, Population: {col_population}, Growth: {col_growth}, "
                       f"Urban Percent: {col_urban_percent}, Urban Pop: {col_urban_pop}.")
    
    cols_to_keep = {}
    if col_year:
        cols_to_keep["Year"] = col_year
    if col_population:
        cols_to_keep["Population"] = col_population
    if col_growth:
        cols_to_keep["Growth_Rate"] = col_growth
    if col_urban_percent:
        cols_to_keep["Urban_Percent"] = col_urban_percent
    if col_urban_pop:
        cols_to_keep["Urban_Population"] = col_urban_pop
        
    df_out = pd.DataFrame()
    for new_col, old_col in cols_to_keep.items():
        if old_col in df.columns:
            df_out[new_col] = df[old_col]
        else:
            df_out[new_col] = np.nan
    
    df_out["Country"] = country_name
    
    # Converte as colunas numéricas removendo vírgulas e símbolos
    for col in ["Population", "Urban_Population"]:
        if col in df_out.columns:
            df_out[col] = (df_out[col]
                           .astype(str)
                           .str.replace(",", "", regex=False)
                           .str.strip())
            df_out[col] = pd.to_numeric(df_out[col], errors='coerce')
    
    for col in ["Urban_Percent", "Growth_Rate"]:
        if col in df_out.columns:
            df_out[col] = (df_out[col]
                           .astype(str)
                           .str.replace("%", "", regex=False)
                           .str.strip())
            df_out[col] = pd.to_numeric(df_out[col], errors='coerce')
    
    if "Year" in df_out.columns:
        df_out["Year"] = pd.to_numeric(df_out["Year"], errors='coerce').astype("Int64")
    
    if "Population" in df_out.columns and "Urban_Population" in df_out.columns:
        df_out["Rural_Population"] = df_out["Population"] - df_out["Urban_Population"]
    else:
        df_out["Rural_Population"] = np.nan
        
    desired_order = ["Country", "Year", "Population", "Growth_Rate", "Urban_Percent", "Urban_Population", "Rural_Population"]
    df_out = df_out[[col for col in desired_order if col in df_out.columns]]
    
    return df_out

async def fetch_country_historical_data(session, country_name, country_url):
    """
    Acessa a página individual do país e extrai a tabela “Population of <país> (2025 and historical)”.
    Em seguida, processa essa tabela para manter somente as colunas de interesse.
    Retorna um DataFrame com os dados históricos (um registro por ano) e com a coluna 'Country'.
    """
    try:
        response = await session.get(country_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Procura por um <h2> cujo texto contenha "Population of" e "historical"
        target_table = None
        for h2 in soup.find_all("h2"):
            txt = h2.get_text().strip()
            if "Population of" in txt and "historical" in txt:
                target_table = h2.find_next("table", {"class": "table-list"})
                break
        
        if target_table is None:
            logger.error(f"Tabela histórica não encontrada para {country_name} ({country_url}).")
            return None
        
        # Lê a tabela com pandas
        df_raw = pd.read_html(str(target_table))[0]
        # Processa a tabela para extrair apenas as colunas desejadas
        df_processed = process_historical_table(df_raw, country_name)
        return df_processed
    except Exception as e:
        logger.error(f"Erro ao extrair dados históricos para {country_name}: {e}")
        return None

async def fetch_all_historical_data():
    """
    A partir do DataFrame principal (obtido via process_population_data),
    acessa o link de cada país e extrai a tabela histórica.
    Retorna um DataFrame consolidado em que cada linha é um registro (Country, Year, ...)
    contendo os dados extraídos (Population, Growth_Rate, Urban_Percent, Urban_Population e Rural_Population).
    """
    session = AsyncHTMLSession()
    try:
        main_df = await process_population_data()
        if main_df is None or main_df.empty:
            logger.error("DataFrame principal com os links dos países está vazio.")
            return None
        
        logger.info(f"Foram encontrados {len(main_df)} países para processar dados históricos.")
        tasks = []
        for _, row in main_df.iterrows():
            country_name = row["Country"]
            country_url = row.get("Country_URL", None)
            if pd.isna(country_url) or not country_url:
                logger.warning(f"URL para {country_name} não encontrada; pulando.")
                continue
            tasks.append(fetch_country_historical_data(session, country_name, country_url))
        
        results = await asyncio.gather(*tasks)
        df_list = [df for df in results if df is not None and not df.empty]
        if df_list:
            df_all = pd.concat(df_list, ignore_index=True)
            return df_all
        else:
            logger.error("Nenhum dado histórico foi extraído.")
            return None
    except Exception as e:
        logger.error(f"Erro crítico ao extrair dados históricos: {e}")
        return None
    finally:
        await session.close()

async def main():
    """
    Função principal que:
      - Extrai o DataFrame principal com os links dos países.
      - Para cada país, acessa seu link e extrai os dados históricos (por ano) 
        mantendo somente as colunas: Population, Growth_Rate, Urban_Percent, Urban_Population;
        calcula Rural_Population (Population - Urban_Population) e inclui o Country.
      - Retorna um dicionário com os DataFrames:
            'Fato_Populacao': dados principais com resumo (incluindo Country_URL)
            'Historico_Pais': dados históricos consolidados (uma linha por país/ano)
    """
    session = AsyncHTMLSession()
    try:
        logger.info("🚀 Iniciando coleta dos dados do Worldometers")
        df_population = await process_population_data()
        logger.info("Dados principais extraídos.")
        
        historical_df = await fetch_all_historical_data()
        
        return {
            'Fato_Populacao': df_population,
            'Historico_Pais': historical_df
        }
    except Exception as e:
        logger.critical(f"Erro crítico: {e}")
        return {
            'Fato_Populacao': pd.DataFrame(),
            'Historico_Pais': pd.DataFrame()
        }
    finally:
        await session.close()

# Execução (em ambiente Jupyter use: await main())
dados = await main()

# Exibe os dados principais (com os links)
if dados['Fato_Populacao'] is not None and not dados['Fato_Populacao'].empty:
    print("=== DADOS POPULACIONAIS PRINCIPAIS ===")
    display(dados['Fato_Populacao'].head(10))
else:
    print("Falha na extração dos dados principais.")

# Exibe o DataFrame consolidado com os dados históricos (um registro por país/ano)
if dados['Historico_Pais'] is not None and not dados['Historico_Pais'].empty:
    print("=== DATAFRAME CONSOLIDADO: UM REGISTRO POR PAÍS/ANO ===")
    display(dados['Historico_Pais'])
    # Opcional: salvar os dados em CSV
    dados['Historico_Pais'].to_csv("historical_population_data.csv", index=False, encoding="utf-8-sig")
    print("Dados históricos salvos em 'historical_population_data.csv'.")
else:
    print("Falha na extração dos dados históricos.")


INFO:__main__:🚀 Iniciando coleta dos dados do Worldometers
INFO:__main__:Tabela principal extraída com 234 linhas e colunas: ['#', 'Country', 'Population (2024)', 'Yearly Change', 'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)', 'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share', 'Country_URL']
INFO:__main__:Dados principais extraídos.
INFO:__main__:Tabela principal extraída com 234 linhas e colunas: ['#', 'Country', 'Population (2024)', 'Yearly Change', 'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)', 'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share', 'Country_URL']
INFO:__main__:Foram encontrados 234 países para processar dados históricos.


=== DADOS POPULACIONAIS PRINCIPAIS ===


,Country,Populacao_Total,Taxa_Crescimento,Urban_Percent,Populacao_Urbana,Populacao_Rural,Country_URL
0,India,1450935791,0.89,37.0,536846243,914089548,https://www.worldometers.info/world-population...
1,China,1419321278,-0.23,66.0,936752043,482569235,https://www.worldometers.info/world-population...
2,United States,345426571,0.57,82.0,283249788,62176783,https://www.worldometers.info/world-population...
3,Indonesia,283487931,0.82,59.0,167257879,116230052,https://www.worldometers.info/world-population...
4,Pakistan,251269164,1.52,34.0,85431516,165837648,https://www.worldometers.info/world-population...
5,Nigeria,232679478,2.10,54.0,125646918,107032560,https://www.worldometers.info/world-population...
6,Brazil,211998573,0.41,91.0,192918701,19079872,https://www.worldometers.info/world-population...
7,Bangladesh,173562364,1.22,42.0,72896193,100666171,https://www.worldometers.info/world-population...
8,Russia,144820423,-0.43,75.0,108615317,36205106,https://www.worldometers.info/world-population...
9,Ethiopia,132059767,2.62,22.0,29053149,103006618,https://www.worldometers.info/world-population...


=== DATAFRAME CONSOLIDADO: UM REGISTRO POR PAÍS/ANO ===


,Country,Year,Population,Growth_Rate,Urban_Percent,Urban_Population,Rural_Population
0,India,2025,1463865525,0.89,37.1,542742539.0,921122986.0
1,India,2024,1450935791,0.89,36.6,530387142.0,920548649.0
2,India,2023,1438069596,0.89,36.0,518239122.0,919830474.0
3,India,2022,1425423212,0.79,35.5,506304869.0,919118343.0
4,India,2020,1402617695,0.98,34.4,483098640.0,919519055.0
...,...,...,...,...,...,...,...
4339,Holy See,1975,733,-0.32,99.3,728.0,5.0
4340,Holy See,1970,745,-3.43,86.4,644.0,101.0
4341,Holy See,1965,887,-0.68,NaN,NaN,NaN
4342,Holy See,1960,918,0.15,98.7,906.0,12.0


Dados históricos salvos em 'historical_population_data.csv'.


## Integração dos DataFrames

In [12]:
import pandas as pd
import numpy as np

# --- 1. Base: Dados Históricos (dados['Historico_Pais']) ---
df_hist = dados['Historico_Pais'].copy()

# Estrutura original: Country | Year | Population | Growth_Rate | Urban_Percent | Urban_Population | Rural_Population
df_hist.rename(columns={
    'Country': 'Entity',
    'Population': 'Populacao_Total',
    'Growth_Rate': 'Taxa_Crescimento',
    # Mantemos o nome 'Urban_Percent' se já estiver bom
    'Urban_Population': 'Populacao_Urbana',
    'Rural_Population': 'Populacao_Rural'
}, inplace=True)

df_hist['Entity'] = df_hist['Entity'].str.lower().str.strip()
df_hist['Year'] = df_hist['Year'].astype(int)

# --- 2. Padronização dos demais DataFrames ---
def padroniza_nome(nome):
    if pd.isna(nome):
        return ""
    return nome.strip().lower()

# df_pib_per_capita -> Entity, Code, Year, PIB_Per_Capita
df_pib_per_capita['Entity'] = df_pib_per_capita['Entity'].apply(padroniza_nome)
df_pib_per_capita['Year'] = df_pib_per_capita['Year'].astype(int)

# df_acesso_educacao -> Entity, Code, Year, Acesso_Educacao
df_acesso_educacao['Entity'] = df_acesso_educacao['Entity'].apply(padroniza_nome)
df_acesso_educacao['Year'] = df_acesso_educacao['Year'].astype(int)

# df_expectativa_vida -> Entity, Code, Year, Expectativa_Vida
df_expectativa_vida['Entity'] = df_expectativa_vida['Entity'].apply(padroniza_nome)
df_expectativa_vida['Year'] = df_expectativa_vida['Year'].astype(int)

# df_taxa_mortalidade -> (renomeia location_name para Entity e year para Year)
df_taxa_mortalidade.rename(columns={'location_name': 'Entity', 'year': 'Year'}, inplace=True)
df_taxa_mortalidade['Entity'] = df_taxa_mortalidade['Entity'].apply(padroniza_nome)
df_taxa_mortalidade['Year'] = df_taxa_mortalidade['Year'].astype(int)

# df_medicos_por_habitante -> (renomeia Location para Entity, Period para Year)
df_medicos_por_habitante.rename(columns={'Location': 'Entity', 'Period': 'Year'}, inplace=True)
df_medicos_por_habitante['Entity'] = df_medicos_por_habitante['Entity'].apply(padroniza_nome)
df_medicos_por_habitante['Year'] = df_medicos_por_habitante['Year'].astype(int)

# df_em_conflito -> Entity, Code, Year, Em_Conflito
df_em_conflito['Entity'] = df_em_conflito['Entity'].apply(padroniza_nome)
df_em_conflito['Year'] = df_em_conflito['Year'].astype(int)

# --- 3. Integração dos Dados para o DataFrame Fato ---
df_fato = df_hist.copy()

# Merge com PIB
df_fato = df_fato.merge(
    df_pib_per_capita[['Entity','Year','PIB_Per_Capita']],
    on=['Entity','Year'],
    how='left'
)

# Merge com Acesso à Educação
df_fato = df_fato.merge(
    df_acesso_educacao[['Entity','Year','Acesso_Educacao']],
    on=['Entity','Year'],
    how='left'
)

# Merge com Expectativa de Vida
df_fato = df_fato.merge(
    df_expectativa_vida[['Entity','Year','Expectativa_Vida']],
    on=['Entity','Year'],
    how='left'
)

# Merge com Taxa de Mortalidade
df_fato = df_fato.merge(
    df_taxa_mortalidade[['Entity','Year','Taxa_Mortalidade']],
    on=['Entity','Year'],
    how='left'
)

# Merge com Médicos por Habitante
df_fato = df_fato.merge(
    df_medicos_por_habitante[['Entity','Year','Medicos_Por_Habitante']],
    on=['Entity','Year'],
    how='left'
)

# Merge com Em_Conflito
df_fato = df_fato.merge(
    df_em_conflito[['Entity','Year','Em_Conflito']],
    on=['Entity','Year'],
    how='left'
)

# Para registros sem Em_Conflito, define 'Baixo' como valor padrão
df_fato['Em_Conflito'] = df_fato['Em_Conflito'].fillna('Baixo')

# --- 4. Integração do novo DataFrame de Religião ---
# Supondo que você já tenha lido o CSV de religião em df_religiao:
df_religiao.rename(columns={"Country or Area": "Entity"}, inplace=True)
df_religiao['Entity'] = df_religiao['Entity'].apply(padroniza_nome)
df_religiao['Year'] = df_religiao['Year'].astype(int)

# Converte a coluna "Value" para numérico (caso necessário)
df_religiao['Value'] = pd.to_numeric(df_religiao['Value'], errors='coerce')

# Exclui a linha de total, se presente, para evitar agregação inadequada
df_religiao = df_religiao[df_religiao['Religion'] != 'Total']

# Aplica o mapeamento de religiões para criar a coluna "Classificacao"

# Agregação para obter a religião predominante por Entity e Year
# Seleciona a linha com maior "Value" (população) para representar a religião predominante
df_religiao_agg = df_religiao.loc[df_religiao.groupby(['Entity','Year'])['Value'].idxmax()]
df_religiao_agg = df_religiao_agg[['Entity','Year','Religion']]
df_religiao_agg.rename(columns={'Religion': 'Religiao_Predominante'}, inplace=True)

# Merge do dado de religião com o DataFrame Fato
df_fato = df_fato.merge(
    df_religiao_agg, on=['Entity','Year'], how='left'
)

# --- 5. Seleção das colunas finais para o DataFrame Fato ---
colunas_finais = [
    'Year','Entity',
    'Populacao_Total','Taxa_Crescimento','Populacao_Urbana','Populacao_Rural',
    'PIB_Per_Capita','Acesso_Educacao','Expectativa_Vida','Taxa_Mortalidade','Medicos_Por_Habitante',
    'Em_Conflito','Religiao_Predominante'
]
df_fato_final = df_fato[colunas_finais].copy()

print("=== Amostra do Fato unificado (dados históricos) ===")
display(df_fato_final.head(15))

# --- 6. Criação das demais Dimensões ---
# Dim_Tempo
dim_tempo = pd.DataFrame({'Ano': sorted(df_fato_final['Year'].unique())})
dim_tempo['Decada'] = dim_tempo['Ano'].apply(lambda x: (x // 10)*10)

# Dim_Local
dim_local = df_fato_final[['Entity']].drop_duplicates().copy()
dim_local.rename(columns={'Entity': 'Pais'}, inplace=True)
dim_local['Continente'] = 'Desconhecido'  # Mapeamento futuro

# Dim_Religiao: Cria a dimensão com Nome_Religiao e Classificacao
dim_religiao = df_religiao[['Religion','Classificacao']].drop_duplicates().rename(
    columns={'Religion': 'Nome_Religiao'}
)

# --- 7. Geração dos CSVs para análise exploratória ---
df_fato_final.to_csv("df_fato_final.csv", index=False, encoding="utf-8-sig")
dim_tempo.to_csv("dim_tempo.csv", index=False, encoding="utf-8-sig")
dim_local.to_csv("dim_local.csv", index=False, encoding="utf-8-sig")
dim_religiao.to_csv("dim_religiao.csv", index=False, encoding="utf-8-sig")

print("CSV gerado para df_fato_final, dim_tempo, dim_local e dim_religiao.")


=== Amostra do Fato unificado (dados históricos) ===


,Year,Entity,Populacao_Total,Taxa_Crescimento,Populacao_Urbana,Populacao_Rural,PIB_Per_Capita,Acesso_Educacao,Expectativa_Vida,Taxa_Mortalidade,Medicos_Por_Habitante,Em_Conflito,Religiao_Predominante
0,2025,india,1463865525,0.89,542742539.0,921122986.0,NaN,NaN,NaN,NaN,NaN,Baixo,NaN
1,2024,india,1450935791,0.89,530387142.0,920548649.0,NaN,NaN,NaN,NaN,NaN,Baixo,NaN
2,2023,india,1438069596,0.89,518239122.0,919830474.0,9160.0630,NaN,72.0026,NaN,NaN,Médio,NaN
3,2022,india,1425423212,0.79,506304869.0,919118343.0,8544.6880,NaN,71.6984,NaN,NaN,Médio,NaN
4,2020,india,1402617695,0.98,483098640.0,919519055.0,7399.5310,34.076544,70.1563,1.402362e+09,7.23,Médio,NaN
5,2015,india,1328024498,1.32,429069459.0,898955039.0,6488.1460,NaN,69.3260,1.325002e+09,7.23,Médio,NaN
6,2010,india,1243481564,1.49,380744554.0,862737010.0,5059.2827,NaN,67.1616,1.233548e+09,NaN,Alto,NaN
7,2005,india,1154676322,1.77,334479406.0,820196916.0,3881.9660,NaN,64.9385,1.133147e+09,NaN,Alto,NaN
8,2000,india,1057922733,1.96,291350282.0,766572451.0,3099.5030,NaN,62.7492,1.032537e+09,NaN,Alto,NaN
9,1995,india,960301044,2.11,255558824.0,704742220.0,2542.4220,NaN,60.6295,9.377291e+08,NaN,Alto,NaN


CSV gerado para df_fato_final, dim_tempo, dim_local e dim_religiao.


## PostgreSQL
### Carregamento de dados para o PostgreSQL

In [13]:
# Ajuste conforme sua conexão local ou remota
USER = "postgres"
PASSWORD = "102030"
HOST = "localhost"
PORT = "5432"
DB_NAME = "world_population"

CONN_STRING = f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}"
engine = create_engine(CONN_STRING)

try:
    with engine.connect() as conn:
        print("Conexão estabelecida com sucesso!")
except Exception as e:
    print(f"Erro ao conectar: {e}")

Conexão estabelecida com sucesso!


In [19]:
df_fato_final.to_sql('Fato_Populacao', engine, if_exists='replace', index=False)
dim_local.to_sql('Dim_Local', engine, if_exists='replace', index=False)
dim_tempo.to_sql('Dim_Tempo', engine, if_exists='replace', index=False)
dim_religiao.to_sql('Dim_Religiao', engine, if_exists='replace', index=False)

457

In [ ]:
create_tables_script = """
DROP TABLE IF EXISTS "Fato_Populacao" CASCADE;
DROP TABLE IF EXISTS "Dim_Religiao" CASCADE;
DROP TABLE IF EXISTS "Dim_Tempo" CASCADE;
DROP TABLE IF EXISTS "Dim_Local" CASCADE;

CREATE TABLE "Dim_Tempo" (
    "ID_Tempo" SERIAL PRIMARY KEY,
    "Ano" INT NOT NULL,
    "Decada" INT NOT NULL
);

CREATE TABLE "Dim_Local" (
    "ID_Local" SERIAL PRIMARY KEY,
    "Pais" VARCHAR(200) NOT NULL,
    "Continente" VARCHAR(200) NOT NULL
);

CREATE TABLE "Dim_Religiao" (
    "ID_Religiao" SERIAL PRIMARY KEY,
    "Nome_Religiao" VARCHAR(200) NOT NULL,
    "Classificacao" VARCHAR(200) NOT NULL
);

-- **Tabela de Fato com chave primária composta**
CREATE TABLE "Fato_Populacao" (
    "Chave_Tempo" INT NOT NULL,
    "Chave_Local" INT NOT NULL,
    "Chave_Religiao" INT, -- Pode ser NULL para casos onde não há classificação religiosa
    "Em_Conflito" VARCHAR(50) NOT NULL,
    "Populacao_Total" BIGINT,
    "Populacao_Urbana" BIGINT,
    "Populacao_Rural" BIGINT,
    "Taxa_Crescimento" FLOAT,
    "Expectativa_Vida" FLOAT,
    "Taxa_Mortalidade" FLOAT,
    "PIB_Per_Capita" FLOAT,
    "Acesso_Educacao" FLOAT,
    "Medicos_Por_Habitante" FLOAT,
    PRIMARY KEY ("Chave_Tempo", "Chave_Local", "Chave_Religiao", "Em_Conflito"),
    FOREIGN KEY ("Chave_Tempo") REFERENCES "Dim_Tempo" ("ID_Tempo"),
    FOREIGN KEY ("Chave_Local") REFERENCES "Dim_Local" ("ID_Local"),
    FOREIGN KEY ("Chave_Religiao") REFERENCES "Dim_Religiao" ("ID_Religiao") ON DELETE SET NULL
);
"""

try:
    with engine.begin() as conn:
        conn.execute(create_tables_script)
    print("Tabelas criadas com sucesso!")
except Exception as e:
    print(f"Erro criando as tabelas: {e}")


## Consultas OLAP + Visualização

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Consulta 1: Distribuição Religiosa e Impactos Socioeconômicos
# Objetivo: Avaliar a relação entre a predominância de determinadas religiões e indicadores como PIB per capita,
# taxa de mortalidade e expectativa de vida.
query1 = """
SELECT R."Nome_Religiao", 
       AVG(F."PIB_Per_Capita") AS media_pib, 
       AVG(F."Expectativa_Vida") AS media_vida, 
       AVG(F."Taxa_Mortalidade") AS media_mortalidade
FROM "Fato_Populacao" F
JOIN "Dim_Religiao" R ON F."Chave_Religiao" = R."ID_Religiao"
GROUP BY R."Nome_Religiao";
"""
df_q1 = pd.read_sql(query1, engine)
print("Consulta 1: Distribuição Religiosa e Impactos Socioeconômicos")
display(df_q1)

# Plot: três subplots para cada indicador
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
sns.barplot(x="Nome_Religiao", y="media_pib", data=df_q1, ax=axs[0], palette="viridis")
axs[0].set_title("PIB per Capita Médio")
axs[0].set_xlabel("Religião")
axs[0].set_ylabel("PIB per Capita")
axs[0].tick_params(axis='x', rotation=45)

sns.barplot(x="Nome_Religiao", y="media_vida", data=df_q1, ax=axs[1], palette="viridis")
axs[1].set_title("Expectativa de Vida Média")
axs[1].set_xlabel("Religião")
axs[1].set_ylabel("Expectativa de Vida (anos)")
axs[1].tick_params(axis='x', rotation=45)

sns.barplot(x="Nome_Religiao", y="media_mortalidade", data=df_q1, ax=axs[2], palette="viridis")
axs[2].set_title("Taxa de Mortalidade Média")
axs[2].set_xlabel("Religião")
axs[2].set_ylabel("Taxa de Mortalidade")
axs[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# -------------------------------------------------------------------

# Consulta 2: Tendências Religiosas e Urbanização
# Objetivo: Examinar se há relação entre a taxa de urbanização e a distribuição de religiões ao longo do tempo.
query2 = """
SELECT T."Ano", 
       R."Nome_Religiao", 
       AVG(CAST(F."Populacao_Urbana" AS FLOAT)/F."Populacao_Total")*100 AS urban_rate
FROM "Fato_Populacao" F
JOIN "Dim_Tempo" T ON F."Chave_Tempo" = T."ID_Tempo"
JOIN "Dim_Religiao" R ON F."Chave_Religiao" = R."ID_Religiao"
GROUP BY T."Ano", R."Nome_Religiao"
ORDER BY T."Ano";
"""
df_q2 = pd.read_sql(query2, engine)
print("Consulta 2: Tendências Religiosas e Urbanização")
display(df_q2.head(15))

# Plot: Linha para cada religião ao longo dos anos
plt.figure(figsize=(10,6))
for relig in df_q2["Nome_Religiao"].unique():
    dados_relig = df_q2[df_q2["Nome_Religiao"] == relig]
    plt.plot(dados_relig["Ano"], dados_relig["urban_rate"], marker='o', label=relig)
plt.title("Taxa de Urbanização por Religião ao Longo do Tempo")
plt.xlabel("Ano")
plt.ylabel("Taxa de Urbanização (%)")
plt.legend(title="Religião", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------

# Consulta 3: Crescimento Populacional e Diversidade Religiosa
# Objetivo: Avaliar se países com maior diversidade religiosa apresentam taxas de crescimento diferentes.
query3 = """
SELECT R."Nome_Religiao", 
       AVG(F."Taxa_Crescimento") AS media_crescimento
FROM "Fato_Populacao" F
JOIN "Dim_Religiao" R ON F."Chave_Religiao" = R."ID_Religiao"
GROUP BY R."Nome_Religiao";
"""
df_q3 = pd.read_sql(query3, engine)
print("Consulta 3: Crescimento Populacional e Diversidade Religiosa")
display(df_q3)

# Plot: Gráfico de barras da taxa de crescimento médio por religião
plt.figure(figsize=(8,5))
sns.barplot(x="Nome_Religiao", y="media_crescimento", data=df_q3, palette="magma")
plt.title("Taxa de Crescimento Médio por Religião")
plt.xlabel("Religião")
plt.ylabel("Taxa de Crescimento (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------

# Consulta 4: Impacto de Conflitos Políticos na População
# Objetivo: Analisar como situações de conflito político afetam métricas como população total, urbanização e PIB per capita.
query4 = """
SELECT F."Em_Conflito", 
       AVG(F."Populacao_Total") AS media_pop_total, 
       AVG(F."Populacao_Urbana") AS media_pop_urbana, 
       AVG(F."PIB_Per_Capita") AS media_pib
FROM "Fato_Populacao" F
GROUP BY F."Em_Conflito";
"""
df_q4 = pd.read_sql(query4, engine)
print("Consulta 4: Impacto de Conflitos Políticos na População")
display(df_q4)

# Plot: Gráfico de barras múltiplas
fig, ax = plt.subplots(figsize=(8,5))
index = range(len(df_q4))
bar_width = 0.25
opacity = 0.8

bar1 = plt.bar([i - bar_width for i in index], df_q4['media_pop_total'], bar_width,
               alpha=opacity, color='b', label='População Total')
bar2 = plt.bar(index, df_q4['media_pop_urbana'], bar_width,
               alpha=opacity, color='g', label='População Urbana')
bar3 = plt.bar([i + bar_width for i in index], df_q4['media_pib'], bar_width,
               alpha=opacity, color='r', label='PIB per Capita')

plt.xlabel('Conflito Político')
plt.ylabel('Média')
plt.title('Métricas Populacionais por Nível de Conflito')
plt.xticks(index, df_q4['Em_Conflito'])
plt.legend()
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------

# Consulta 5: Taxa de Mortalidade em Regiões por Classificação Religiosa e Continente
# Objetivo: Correlacionar a taxa de mortalidade com a classificação religiosa em diferentes continentes.
query5 = """
SELECT L."Continente", 
       R."Nome_Religiao", 
       AVG(F."Taxa_Mortalidade") AS media_mortalidade
FROM "Fato_Populacao" F
JOIN "Dim_Local" L ON F."Chave_Local" = L."ID_Local"
JOIN "Dim_Religiao" R ON F."Chave_Religiao" = R."ID_Religiao"
GROUP BY L."Continente", R."Nome_Religiao";
"""
df_q5 = pd.read_sql(query5, engine)
print("Consulta 5: Taxa de Mortalidade por Região e Religião")
display(df_q5)

# Plot: Usando seaborn para criar um gráfico de barras agrupadas
plt.figure(figsize=(10,6))
sns.barplot(x="Continente", y="media_mortalidade", hue="Nome_Religiao", data=df_q5, palette="coolwarm")
plt.title("Taxa de Mortalidade por Continente e Religião")
plt.xlabel("Continente")
plt.ylabel("Taxa de Mortalidade")
plt.legend(title="Religião", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------

# Consulta 6: PIB per Capita e Indicadores de Saúde por Religião
# Objetivo: Comparar o PIB per capita com indicadores de saúde (Expectativa de Vida e Taxa de Mortalidade) por religião.
query6 = """
SELECT R."Nome_Religiao", 
       AVG(F."PIB_Per_Capita") AS media_pib, 
       AVG(F."Expectativa_Vida") AS media_vida, 
       AVG(F."Taxa_Mortalidade") AS media_mortalidade
FROM "Fato_Populacao" F
JOIN "Dim_Religiao" R ON F."Chave_Religiao" = R."ID_Religiao"
GROUP BY R."Nome_Religiao";
"""
df_q6 = pd.read_sql(query6, engine)
print("Consulta 6: PIB per Capita e Indicadores de Saúde por Religião")
display(df_q6)

# Plot: Scatter plot com PIB no eixo x, Expectativa de Vida no eixo y e tamanho da bolha proporcional à Taxa de Mortalidade
plt.figure(figsize=(8,6))
plt.scatter(df_q6['media_pib'], df_q6['media_vida'], s=df_q6['media_mortalidade']*0.001, alpha=0.7, color='purple')
for i, relig in enumerate(df_q6['Nome_Religiao']):
    plt.text(df_q6['media_pib'][i], df_q6['media_vida'][i], relig, fontsize=10)
plt.xlabel("PIB per Capita Médio")
plt.ylabel("Expectativa de Vida Média (anos)")
plt.title("Relação entre PIB, Expectativa de Vida e Taxa de Mortalidade por Religião")
plt.tight_layout()
plt.show()
